In [17]:
from collections import defaultdict

def test_model(model, training_data, test_data):
    '''
    Function to test `model` on `test_data` given `training_data`
    `test_data` is a list of (text, label) tuples
    '''
    model.train(training_data)
    
    # confusion matrix to store results used for calculating precision and recall
    # confusion_matrix[gold_label][model_label] = number of occurrences
    confusion_matrix = defaultdict(lambda: defaultdict(lambda: 0))
    
    for text, gold_label in test_data:
        model_label = model.identify(text)[0][1]        
        confusion_matrix[gold_label][model_label] += 1
    
    # compute individual class matrices
    class_matrix = {}
    num_results = sum([sum(confusion_matrix[auth].values()) for auth in confusion_matrix])
    
    for auth in confusion_matrix:
        class_matrix[auth] = {
            'tp': confusion_matrix[auth][auth],
            'fp': sum([confusion_matrix[i][auth] for i in confusion_matrix if i != auth]),
            'fn': sum(confusion_matrix[auth].values()) - confusion_matrix[auth][auth]
        }
        class_matrix[auth]['tn'] = num_results - sum(class_matrix[auth].values())
        
    # compute a pooled matrix
    pooled_matrix = {
        'tp': sum([class_matrix[auth]['tp'] for auth in class_matrix]),
        'fp': sum([class_matrix[auth]['fp'] for auth in class_matrix]),
        'tn': sum([class_matrix[auth]['tn'] for auth in class_matrix]),
        'fn': sum([class_matrix[auth]['fn'] for auth in class_matrix])
    }

    # macro precision and recall
    class_precision = {}
    class_recall = {}
    
    for auth in class_matrix:
        try:
            class_precision[auth] = class_matrix[auth]['tp'] / (class_matrix[auth]['tp'] + class_matrix[auth]['fp'])
        except ZeroDivisionError:
            # classifier assigned negative to all cases
            # signal this with negative infinite precision
            class_precision[auth] = float('-inf')
        try:
            class_recall[auth] = class_matrix[auth]['tp'] / (class_matrix[auth]['tp'] + class_matrix[auth]['fn'])
        except ZeroDivisionError:
            # there were no positive cases in the input
            # signal this with infinite precision
            class_precision[auth] = float('inf')
    
    macro_precision = sum(class_precision.values()) / len(class_precision)
    macro_recall = sum(class_recall.values()) / len(class_recall)
    
    # compute micro precision and recall
    micro_precision = pooled_matrix['tp'] / (pooled_matrix['tp'] + pooled_matrix['fp'])
    micro_recall = pooled_matrix['tp'] / (pooled_matrix['tp'] + pooled_matrix['fn'])
    
    # compute F1 scores
    macro_f1 = (2 * macro_precision * macro_recall) / (macro_precision + macro_recall)
    micro_f1 = (2 * micro_precision * micro_recall) / (micro_precision + micro_recall)
    
    # return F1 scores and confusion matrix
    return macro_f1, micro_f1, confusion_matrix

In [15]:
from models.ensemble_model import Ensemble
from models.compression_model import CompressionModel
from utils.data_reader import read_data

# initialize ensemble model
model = Ensemble()

# read in the data
data = read_data()
# training data is everything but the iliad
# NOTE: should almost certainly change this, this is just to get us started
training_data = { auth: texts[1:] for auth, texts in data.items() }

# make test data the iliad split on different sections
test_data = []
i = 0
for auth in data:
    iliad = data[auth][0]
    for section in iliad.split('\n\n'):
        test_data.append((section, auth))

test_model(model, training_data, test_data)

{'Chapman': 110983, 'Cowper': 118206, 'Dryden': 109204, 'Pope': 110674}
{'Chapman': 146925, 'Cowper': 155597, 'Dryden': 144551, 'Pope': 146203}
{'Chapman': 90828, 'Cowper': 98267, 'Dryden': 88919, 'Pope': 90221}
{'Chapman': 107455, 'Cowper': 115757, 'Dryden': 105568, 'Pope': 106852}
{'Chapman': 165004, 'Cowper': 173978, 'Dryden': 163223, 'Pope': 164647}
{'Chapman': 104530, 'Cowper': 112549, 'Dryden': 102704, 'Pope': 103866}
{'Chapman': 76231, 'Cowper': 82696, 'Dryden': 74497, 'Pope': 75604}
{'Chapman': 93539, 'Cowper': 101073, 'Dryden': 91910, 'Pope': 92762}
{'Chapman': 122386, 'Cowper': 130725, 'Dryden': 119217, 'Pope': 120189}
{'Chapman': 90461, 'Cowper': 97813, 'Dryden': 88834, 'Pope': 89520}
{'Chapman': 140247, 'Cowper': 148997, 'Dryden': 138391, 'Pope': 139469}
{'Chapman': 84759, 'Cowper': 90359, 'Dryden': 83147, 'Pope': 84643}
{'Chapman': 138873, 'Cowper': 146377, 'Dryden': 136651, 'Pope': 138473}
{'Chapman': 82092, 'Cowper': 89487, 'Dryden': 81290, 'Pope': 81674}
{'Chapman': 128

(nan,
 0.35135135135135137,
 defaultdict(<function __main__.test_model.<locals>.<lambda>()>,
             {'Chapman': defaultdict(<function __main__.test_model.<locals>.<lambda>.<locals>.<lambda>()>,
                          {'Dryden': 24,
                           'Chapman': 0,
                           'Cowper': 0,
                           'Pope': 0}),
              'Cowper': defaultdict(<function __main__.test_model.<locals>.<lambda>.<locals>.<lambda>()>,
                          {'Dryden': 24,
                           'Chapman': 0,
                           'Cowper': 0,
                           'Pope': 0}),
              'Dryden': defaultdict(<function __main__.test_model.<locals>.<lambda>.<locals>.<lambda>()>,
                          {'Dryden': 2, 'Chapman': 0, 'Cowper': 0, 'Pope': 0}),
              'Pope': defaultdict(<function __main__.test_model.<locals>.<lambda>.<locals>.<lambda>()>,
                          {'Pope': 24,
                           'Chapman': 0,
